In [1]:
import statsmodels.api as sm
import pandas as pd

In [ ]:
df_1 = pd.read_excel(r"iv_data.xlsx", usecols=["log_content length"])
df_2 = pd.read_excel(r"control_variable.xlsx")
df_2.drop(columns=["nid"], inplace=True)
df = pd.concat([df_1, df_2], axis=1)
df.head(1)

In [3]:
df_1 = pd.read_excel(r"result_ambiguity.xlsx")
df_1.drop(columns=["number"], inplace=True)

df_2 = pd.read_excel(r"result_greenwashing.xlsx")
df_2.drop(columns=["number"], inplace=True)

df_3 = pd.read_excel(r"result_readability.xlsx")
df_3.drop(columns=["number"], inplace=True)

df_4 = pd.read_excel(r"result_subjectivity.xlsx")
df_4.drop(columns=["number"], inplace=True)

df_5 = pd.read_excel(r"result_timeliness.xlsx")
df_5.drop(columns=["number"], inplace=True)

# 1. Predict Variable by IV

In [ ]:
X = df[[df.columns[0], df.columns[1], df.columns[2], df.columns[3]]]
X = sm.add_constant(X)

In [ ]:
result = {}
for model in df_5.columns:
    Y = df_5[model]
    first_stage = sm.OLS(Y, X).fit()
    result[model] = first_stage.resid.tolist()


In [ ]:
df_resident = pd.DataFrame(result)
df_resident.head(1)

# 2. Outcome Model 加上 residual

In [ ]:
df_car = pd.read_excel(r"cumulated_abnormal_return.xlsx")
df_car.drop(columns=["number", "CAR [0,3]", "CAR [0,4]"], inplace=True)
df_car.head(1)

In [ ]:
def endogeneity_test(df, df_car, df_resident):
    result = {}
    df_use = pd.concat([df, df_resident], axis=1)
    for model in df_use.columns[4:]:
        X2 = df_use[[df_use.columns[1], df_use.columns[2], df_use.columns[3], model]]
        X2 = sm.add_constant(X2)
        each_model_result = []
        for car in df_car.columns:
            Y2 = df_car[car]
            second_stage = sm.OLS(Y2, X2).fit()
            resident_pvalue = second_stage.pvalues[model] 
            if resident_pvalue < 0.05:
                resident_pvalue = str(round(resident_pvalue, 4))+"***"
            else:
                resident_pvalue = round(resident_pvalue, 4)
            each_model_result.append(resident_pvalue)
        
        result[model] = each_model_result
    
    df_endogeneity = pd.DataFrame(result)
    df_endogeneity.to_excel(f"{model}_endogeneity.xlsx", index=False)

In [25]:
endogeneity_test(df, df_car, df_resident)